### Importing relevent libraries


In [100]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer




In [101]:
# Load the dataset
data = pd.read_csv("../Datasets/Marketing campaign dataset.csv")
# Display the DataFrame
print(data)


       campaign_item_id  no_of_days        time  ext_service_id  \
0                  2733           7  2022-05-01             128   
1                  2733           8  2022-05-02              16   
2                  2733           9  2022-05-03             128   
3                  2733          10  2022-05-04             128   
4                  2733          11  2022-05-05               4   
...                 ...         ...         ...             ...   
72607              3567          11  2022-11-28               4   
72608              3567          12  2022-11-29             128   
72609              3567          13  2022-11-30             128   
72610              3567          14  2022-12-01               4   
72611              3567          15  2022-12-02              16   

      ext_service_name  creative_id  creative_width  creative_height  \
0         Facebook Ads         1000           300.0            250.0   
1                DV360         1000           300.0

In [102]:
print(data.dtypes)

campaign_item_id         int64
no_of_days               int64
time                    object
ext_service_id           int64
ext_service_name        object
creative_id              int64
creative_width         float64
creative_height        float64
search_tags             object
template_id            float64
landing_page            object
advertiser_id            int64
advertiser_name         object
network_id               int64
approved_budget        float64
advertiser_currency     object
channel_id               int64
channel_name            object
max_bid_cpm            float64
network_margin         float64
campaign_budget_usd    float64
impressions              int64
clicks                   int64
stats_currency          object
currency_code           object
exchange_rate            int64
media_cost_usd         float64
position_in_content    float64
unique_reach           float64
total_reach            float64
search_tag_cat          object
cmi_currency_code       object
timezone

In [103]:
# Select relevant features
selected_columns = [
    'campaign_item_id', 'advertiser_id', 'advertiser_name', 'creative_id', 'creative_width', 
    'creative_height', 'search_tags', 'template_id', 'channel_id', 'channel_name', 
    'campaign_budget_usd', 'impressions', 'clicks', 'media_cost_usd', 'weekday_cat', 
    'keywords', 'time', 'ext_service_name'
]

# Select the relevant columns
data = data[selected_columns]

# Convert 'time' to datetime format
data['time'] = pd.to_datetime(data['time'])

In [104]:
# Initialize LabelEncoder
label_encoder = LabelEncoder()

# List of categorical columns to encode
categorical_columns = ['ext_service_name', 'channel_name', 'weekday_cat', 'search_tags', 'advertiser_name', 'keywords']

# Loop through the categorical columns and encode them
for col in categorical_columns:
    data[col] = label_encoder.fit_transform(data[col])


# Display the DataFrame after encoding
print(data.head())


   campaign_item_id  advertiser_id  advertiser_name  creative_id  \
0              2733           4756               42         1000   
1              2733           4756               42         1000   
2              2733           4756               42         1000   
3              2733           4756               42         1000   
4              2733           4756               42         1000   

   creative_width  creative_height  search_tags  template_id  channel_id  \
0           300.0            250.0            2         90.0          32   
1           300.0            250.0            0         90.0           8   
2           300.0            250.0            1         90.0           8   
3           300.0            250.0            0         90.0          64   
4           300.0            250.0            0         90.0          32   

   channel_name  campaign_budget_usd  impressions  clicks  media_cost_usd  \
0             1           652.173913          837       8

In [105]:
data['creative_dimension'] = data['creative_height'] * data['creative_width']

data = data.drop(columns=['creative_height', 'creative_width'])


In [106]:
# Check for null values in the dataset
null_values = data.isnull().sum()

# Display columns with null values (if any)
print("Null values in each column:\n", null_values)

# To check if there are any rows with missing values in the entire dataset
print("Total rows with any missing values:", data.isnull().any(axis=1).sum())


Null values in each column:
 campaign_item_id          0
advertiser_id             0
advertiser_name           0
creative_id               0
search_tags               0
template_id            3412
channel_id                0
channel_name              0
campaign_budget_usd       0
impressions               0
clicks                    0
media_cost_usd            0
weekday_cat               0
keywords                  0
time                      0
ext_service_name          0
creative_dimension     3412
dtype: int64
Total rows with any missing values: 3412


In [107]:
# Fill null values in specific columns with 0
columns_to_fill = ['creative_dimension']
data[columns_to_fill] = data[columns_to_fill].fillna(0)

# Verify the changes
print(data[columns_to_fill].isnull().sum())  # Should show 0 for all these columns
print(data.head())  # To see the updated dataset

creative_dimension    0
dtype: int64
   campaign_item_id  advertiser_id  advertiser_name  creative_id  search_tags  \
0              2733           4756               42         1000            2   
1              2733           4756               42         1000            0   
2              2733           4756               42         1000            1   
3              2733           4756               42         1000            0   
4              2733           4756               42         1000            0   

   template_id  channel_id  channel_name  campaign_budget_usd  impressions  \
0         90.0          32             1           652.173913          837   
1         90.0           8             3           652.173913         2634   
2         90.0           8             3           652.173913         2135   
3         90.0          64             4           652.173913         2327   
4         90.0          32             1           652.173913         1538   

   clic

In [108]:
data.describe()

,campaign_item_id,advertiser_id,advertiser_name,creative_id,search_tags,template_id,channel_id,channel_name,campaign_budget_usd,impressions,clicks,media_cost_usd,weekday_cat,keywords,time,ext_service_name,creative_dimension
count,72612.000000,72612.000000,72612.000000,72612.000000,72612.00000,69200.000000,72612.000000,72612.000000,72612.000000,72612.000000,72612.000000,72612.000000,72612.000000,72612.000000,72612,72612.000000,72612.000000
mean,3130.143282,6195.862213,23.330882,7450.124842,2.48828,83.082659,21.847780,2.001474,617.630284,1370.161847,52.696386,11.819999,0.140307,58.433537,2022-09-26 18:40:05.949429760,0.999931,63806.602215
min,2733.000000,4756.000000,0.000000,1000.000000,0.00000,23.000000,1.000000,0.000000,2.452316,511.000000,2.000000,0.000000,0.000000,0.000000,2022-05-01 00:00:00,0.000000,0.000000
25%,3148.000000,6319.000000,17.000000,3725.000000,1.00000,90.000000,4.000000,1.000000,205.994550,526.000000,5.000000,0.435791,0.000000,29.000000,2022-08-20 00:00:00,0.000000,75000.000000
50%,3173.000000,6385.000000,25.000000,7855.000000,2.00000,90.000000,8.000000,2.000000,377.656676,577.000000,8.000000,1.709001,0.000000,58.000000,2022-10-11 00:00:00,1.000000,75000.000000
75%,3202.000000,6394.000000,33.000000,10995.000000,4.00000,90.000000,32.000000,3.000000,755.313351,816.000000,13.000000,8.276369,0.000000,88.000000,2022-11-05 00:00:00,2.000000,75000.000000
max,3960.000000,6490.000000,43.000000,15605.000000,5.00000,93.000000,64.000000,4.000000,39559.896155,153959.000000,31807.000000,2295.028945,1.000000,117.000000,2022-12-10 00:00:00,2.000000,75000.000000
std,142.154918,387.864576,11.997667,4062.384982,1.70249,20.847618,23.759286,1.413565,1354.606619,3702.699962,377.308103,48.370659,0.347308,34.024005,NaN,0.815903,26724.941835


In [109]:
data["template_id"].describe()
data["template_id"].isnull().sum()  # Count of missing values
print(data["template_id"].unique())


[90. 23. 92. 89. 93. nan]


In [110]:
data["template_id"] = data["template_id"].fillna(data["template_id"].mean())

In [111]:
data.to_csv("../Datasets/Dataset.csv",  index=False)